In [6]:
#Importing required libraries
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [2]:
# Load the dataset
df = pd.read_csv(Path('./Resources/mental_health_data_revised.csv'))
df.head(10)

,SELF_EMPLOYED,NUM_OF_EMPLOYEES,TECH_COMPANY?,TECH_ROLE?,MENTAL_HEALTH_COVERAGE?,MENTAL_HEALTH_OPTIONS_UNDER_COVERAGE,EMPLOYER_DISCUSSED_MENTAL_HEALTH_FORMALLY?,EMPLOYER_OFFER_MENTAL_HEALTH_RESOURCES?,ANONMITY PROTECTED BY EMPLOYER IF USE OF MENTAL HEALTH/SUB ABUSE RESOURCES ARE TAKEN?,LEVEL OF DIFFICULTY ASKING FOR MENTAL HEALTH LEAVE,...,DOES_MENTAL_HEALTH_ISSUE_INTERFERE_WITH_WORK_BEING_TREATED_EFFECTIVELY?,DOES_MENTAL_HEALTH_ISSUE_INTERFERE_WITH_WORK_NOT_BEING_TREATED_EFFECTIVELY?,AGE,GENDER,COUNTRY_LIVING_IN,STATE_LIVING_IN,COUNTRY_WORKING_IN,STATE_WORKING_IN,WORK_POSITION,WFH
0,0,26-100,1.0,NaN,Not eligible for coverage / N/A,NaN,No,No,I don't know,Very easy,...,Not applicable to me,Not applicable to me,39,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Sometimes
1,0,25-Jun,1.0,NaN,No,Yes,Yes,Yes,Yes,Somewhat easy,...,Rarely,Sometimes,29,male,United States of America,Illinois,United States of America,Illinois,Back-end Developer|Front-end Developer,Never
2,0,25-Jun,1.0,NaN,No,NaN,No,No,I don't know,Neither easy nor difficult,...,Not applicable to me,Not applicable to me,38,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Always
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Sometimes,Sometimes,43,male,United Kingdom,NaN,United Kingdom,NaN,Supervisor/Team Lead,Sometimes
4,0,25-Jun,0.0,1.0,Yes,Yes,No,No,No,Neither easy nor difficult,...,Sometimes,Sometimes,43,Female,United States of America,Illinois,United States of America,Illinois,Executive Leadership|Supervisor/Team Lead|Dev ...,Sometimes
5,0,More than 1000,1.0,NaN,Yes,I am not sure,No,Yes,Yes,Somewhat easy,...,Not applicable to me,Often,42,Male,United Kingdom,NaN,United Kingdom,NaN,DevOps/SysAdmin|Support|Back-end Developer|Fro...,Sometimes
6,0,26-100,1.0,NaN,I don't know,No,No,No,I don't know,Somewhat easy,...,Not applicable to me,Not applicable to me,30,M,United States of America,Tennessee,United States of America,Tennessee,Back-end Developer,Sometimes
7,0,More than 1000,1.0,NaN,Yes,Yes,No,Yes,Yes,Very easy,...,Sometimes,Often,37,female,United States of America,Virginia,United States of America,Virginia,Dev Evangelist/Advocate|Back-end Developer,Always
8,0,26-100,0.0,1.0,I don't know,No,No,No,I don't know,Very difficult,...,Rarely,Often,44,Female,United States of America,California,United States of America,California,Support|Back-end Developer|One-person shop,Sometimes
9,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Rarely,Often,30,Male,United States of America,Kentucky,United States of America,Kentucky,One-person shop|Front-end Developer|Back-end D...,Always


In [3]:
df.columns

Index(['SELF_EMPLOYED', 'NUM_OF_EMPLOYEES', 'TECH_COMPANY?', 'TECH_ROLE?',
       'MENTAL_HEALTH_COVERAGE?', 'MENTAL_HEALTH_OPTIONS_UNDER_COVERAGE',
       'EMPLOYER_DISCUSSED_MENTAL_HEALTH_FORMALLY?',
       'EMPLOYER_OFFER_MENTAL_HEALTH_RESOURCES?',
       'ANONMITY PROTECTED BY EMPLOYER IF USE OF MENTAL HEALTH/SUB ABUSE RESOURCES ARE TAKEN?',
       'LEVEL OF DIFFICULTY ASKING FOR MENTAL HEALTH LEAVE',
       'DISCUSSING MENTAL HEALTH WITH EMPLOYER WOULD HAVE NEG CONSEQ?',
       'DISCUSSING PHYSICAL HEALTH WITH EMPLOYER WOULD HAVE NEG CONSEQ?',
       'COMFORTABLE DISCUSSING MENTAL HEALTH WITH COWORKERS?',
       'COMFORTABLE DISCUSSING MENTAL HEALTH WITH SUPERVISOR?',
       'DOES_EMPLOYER_TAKE_MENTAL_HEALTH_AS_SERIOUSLY_AS_PHYSICAL_HEALTH?',
       'HAVE CO-WORKERS RECEIVED NEGATIVE CONSEQ FOR MENTAL HEALTH?',
       'MEDICAL_COVERAGE_INCLUDING_MENTAL_HEALTH_TREATMENT?',
       'KNOWLEDGE OF MENTAL HEALTH RESOURCES TO SEEK FOR HELP?',
       'DO YOU REVEAL TO CLIENTS/BUSINESSES D

In [4]:
df.shape

(1433, 63)

In [5]:
df.info

<bound method DataFrame.info of       SELF_EMPLOYED NUM_OF_EMPLOYEES  TECH_COMPANY?  TECH_ROLE?  \
0                 0           26-100            1.0         NaN   
1                 0           25-Jun            1.0         NaN   
2                 0           25-Jun            1.0         NaN   
3                 1              NaN            NaN         NaN   
4                 0           25-Jun            0.0         1.0   
...             ...              ...            ...         ...   
1428              1              NaN            NaN         NaN   
1429              1              NaN            NaN         NaN   
1430              0          100-500            1.0         NaN   
1431              0          100-500            0.0         1.0   
1432              0          100-500            1.0         NaN   

              MENTAL_HEALTH_COVERAGE? MENTAL_HEALTH_OPTIONS_UNDER_COVERAGE  \
0     Not eligible for coverage / N/A                                  NaN   
1      

In [19]:
df['NUM_OF_EMPLOYEES'] = df['NUM_OF_EMPLOYEES'].astype(int)
df

ValueError: invalid literal for int() with base 10: '26-100'

In [ ]:
df.dropna()